In [21]:
import pandas as pd
from pyspark.sql import SparkSession, functions
from datetime import date

In [2]:
def format_date(date):
    temp = date.split("/")
    temp.insert(0, temp[2])
    temp = list(map(int,temp[:3]))
    return temp

big_list = []
with open('YelpData.txt', 'rb') as f:
    data = f.readlines()
for partial_data in data:
    df = pd.read_json(partial_data)
    for column in df.columns:
        temp = list(df[column])
        temp.insert(0,column)
        big_list.append(temp)
new_df = pd.DataFrame(big_list,columns=["key", "api", "restaurant","date", "review", "rating", "num_votes"])
new_df.date = new_df.date.map(lambda x: date(*format_date(x)))

In [75]:
spark_df.show()

+--------------------+----+------------+----------+--------------------+------+---------+
|                 key| api|  restaurant|      date|              review|rating|num_votes|
+--------------------+----+------------+----------+--------------------+------+---------+
|1nNArxJdbKsBTDiyl...|Yelp| Sushi Osaka|2019-02-01|Note to future se...|     1|        1|
|6dn6Z2ZxuN4EvwPOo...|Yelp| Sushi Osaka|2018-11-27|I've been here tw...|     2|        1|
|7rfumZn_RU5VjaJvC...|Yelp| Sushi Osaka|2019-01-28|Great sushi, pric...|     5|        1|
|D15UsRq7j1ttFKbcX...|Yelp| Sushi Osaka|2018-09-05|The sushi is deli...|     5|       19|
|HFR1zIJr5ES_3e5nW...|Yelp| Sushi Osaka|2018-04-20|Ate here twice in...|     5|       19|
|W6nEFj0l6D1T86Vwm...|Yelp| Sushi Osaka|2018-04-28|I was hesitant in...|     2|        5|
|WCIxWyhNgzhaUszTr...|Yelp| Sushi Osaka|2018-11-10|I went to Sushi O...|     2|        1|
|fGGmLCP2Fz_V5FPRK...|Yelp| Sushi Osaka|2018-06-18|Best and cheapest...|     5|       16|
|l7oULh_4r

In [57]:
spark = SparkSession.builder.appName('restaurant_reviews').getOrCreate()
spark_df = spark.createDataFrame(new_df)
# spark_df.write.saveAsTable("yelp")

+--------------------+---------+
|          restaurant|num_votes|
+--------------------+---------+
|        Gorgers Subs|      140|
|            Tamarind|      106|
|     Napoli Pizzeria|       48|
|Luna Inspired Str...|       90|
|Applebee’s Grill ...|       27|
|  Buffalo Wild Wings|       31|
|              Subway|        1|
|       Stella’s Barn|       27|
|Capital State Kit...|       20|
|         Sushi Osaka|        9|
|     Gangnam Station|       31|
|Spring Buffet & G...|       37|
| Dapper’s Restaurant|        6|
|Taste of Thai Exp...|      166|
|           Taco Bell|       16|
|         Pizza Aroma|       55|
|       Taste of Thai|      180|
|         Atrium Cafe|        3|
|Little Caesars Pizza|        3|
|         Nikki Green|       33|
+--------------------+---------+
only showing top 20 rows



In [83]:
def rating_counts(df):
    df = df.select(["restaurant", "rating"])
    for i in range(1,6):
        df = df.withColumn("rating_" + str(i), functions.when(functions.col("rating") == i,1).otherwise(0))
    return df.groupBy("restaurant").sum()

def get_review_text_date_api(df_yelp, df_zomato, rest_name):
    yelp = df_yelp.where(df_yelp.restaurant == rest_name).select(["review", "date", "api"])
    zomato = df_zomato.where(df_zomato.restaurant == rest_name).select(["review", "date", "api"])
    return yelp.union(zomato)

def get_restaurant_counts(df):
    return df.groupBy("restaurant").count()
def get_vote_counts(df):
    return spark_df.select(["restaurant", "num_votes"]).groupBy("restaurant").count().withColumnRenamed("count", "num_votes").sho